In [24]:
from pyspark.sql import SparkSession

In [25]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [26]:
from pyspark.ml.regression import LinearRegression

In [27]:
data = spark.read.csv('Spark_for_Machine_Learning/0.Linear_Regression/Ecommerce_Customers.csv', inferSchema= True, header= True)

In [28]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [29]:
for item in data.head(2)[1]:
    print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


# Preparing the data

In [30]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [31]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [32]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'], outputCol='features')

In [33]:
output = assembler.transform(data)

In [34]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [35]:
final_data = output.select('features', 'Yearly Amount Spent' )

In [36]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [37]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

# Performing the Linear Regression

In [38]:
# Lets create the linera regression model
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

In [39]:
lr_model = lr.fit(train_data)

In [40]:
test_results = lr_model.evaluate(test_data)

In [42]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -6.001357985642073|
| -4.202444565036501|
| -7.627772713952709|
|-1.3669091761604477|
|-4.3095920381422275|
|  -8.07599140160903|
| -5.515503316234117|
| -4.925037439069058|
| -6.043906004886708|
|  3.368877772464714|
| 17.570167475895005|
| -7.187411450719878|
| -2.506698278052397|
| 1.7534187462754858|
|-4.9590092060291795|
|  -11.7294799865399|
| -4.496159673559987|
|-18.165622838705588|
|  -2.88823825567124|
|-2.2318247248532543|
+-------------------+
only showing top 20 rows



In [43]:
test_results.rootMeanSquaredError

9.140644552602566

In [44]:
test_results.r2

0.9851594758160814

# Deploying the model

In [48]:
unlabeled_data = test_data.select('features')

In [49]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.8794843441274...|
|[31.1280900496166...|
|[31.2606468698795...|
|[31.4252268808548...|
|[31.4474464941278...|
|[31.5147378578019...|
|[31.5171218025062...|
|[31.5257524169682...|
|[31.5316044825729...|
|[31.6005122003032...|
|[31.7207699002873...|
|[31.7216523605090...|
|[31.7366356860502...|
|[31.7656188210424...|
|[31.8093003166791...|
|[31.8124825597242...|
|[31.8164283341993...|
|[31.8530748017465...|
|[31.8627411090001...|
+--------------------+
only showing top 20 rows

